# Lumeo - Dataset Exploration

This notebook explores the LOL (LOw-Light) dataset for the low-light image enhancement project.

**Dataset Structure:**
- Training: 485 paired images (low → high)
- Validation: 15 paired images
- Test: 335 unpaired low-light images (no ground truth)

## 1. Setup & Mount Drive

In [ ]:
# Mount Google Drive (upload dataset to Drive first)
from google.colab import drive
drive.mount('/content/drive')

# Set dataset paths - UPDATE THESE TO YOUR DRIVE PATHS
DATASET_ROOT = '/content/drive/MyDrive/Lumeo/datasets'

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
import random

# Paths
TRAIN_LOW = os.path.join(DATASET_ROOT, 'LOLdataset/our485/low')
TRAIN_HIGH = os.path.join(DATASET_ROOT, 'LOLdataset/our485/high')
VAL_LOW = os.path.join(DATASET_ROOT, 'LOLdataset/eval15/low')
VAL_HIGH = os.path.join(DATASET_ROOT, 'LOLdataset/eval15/high')
TEST_ROOT = os.path.join(DATASET_ROOT, 'Test/Test')

print(f"Train Low: {TRAIN_LOW}")
print(f"Train High: {TRAIN_HIGH}")

## 2. Dataset Statistics

In [ ]:
def count_images(path):
    """Count PNG images in directory"""
    if not os.path.exists(path):
        return 0
    return len([f for f in os.listdir(path) if f.endswith('.png')])

def get_image_names(path):
    """Get sorted list of image names"""
    if not os.path.exists(path):
        return []
    return sorted([f for f in os.listdir(path) if f.endswith('.png')])

# Count images
print("=== Dataset Statistics ===")
print(f"Training Low:  {count_images(TRAIN_LOW)} images")
print(f"Training High: {count_images(TRAIN_HIGH)} images")
print(f"Validation Low:  {count_images(VAL_LOW)} images")
print(f"Validation High: {count_images(VAL_HIGH)} images")

# Count test images
test_dirs = ['DICM', 'Fusion', 'LIME', 'MEF', 'NPE', 'VV', 'low']
print("\n=== Test Dataset ===")
total_test = 0
for d in test_dirs:
    path = os.path.join(TEST_ROOT, d)
    count = count_images(path)
    if count > 0:
        print(f"  {d}: {count} images")
        total_test += count
print(f"  Total Test: {total_test} images")

In [ ]:
# Verify pairing: check that low and high have matching filenames
train_low_names = set(get_image_names(TRAIN_LOW))
train_high_names = set(get_image_names(TRAIN_HIGH))

matched = train_low_names.intersection(train_high_names)
unmatched_low = train_low_names - train_high_names
unmatched_high = train_high_names - train_low_names

print(f"\n=== Pairing Verification ===")
print(f"Matched pairs: {len(matched)}")
print(f"Unmatched in low: {len(unmatched_low)}")
print(f"Unmatched in high: {len(unmatched_high)}")

if len(unmatched_low) > 0:
    print(f"  Missing high counterparts: {list(unmatched_low)[:5]}...")
if len(unmatched_high) > 0:
    print(f"  Missing low counterparts: {list(unmatched_high)[:5]}...")

## 3. Visualize Sample Pairs

In [ ]:
def load_image(path):
    """Load image and convert to RGB"""
    return np.array(Image.open(path).convert('RGB'))

def display_pairs(low_dir, high_dir, num_pairs=5, seed=42):
    """Display low-light and corresponding high-light pairs"""
    random.seed(seed)
    
    # Get common image names
    low_names = set(get_image_names(low_dir))
    high_names = set(get_image_names(high_dir))
    common = sorted(list(low_names.intersection(high_names)))
    
    # Sample random pairs
    samples = random.sample(common, min(num_pairs, len(common)))
    
    fig, axes = plt.subplots(num_pairs, 2, figsize=(12, 4*num_pairs))
    if num_pairs == 1:
        axes = [axes]
    
    for idx, name in enumerate(samples):
        low_img = load_image(os.path.join(low_dir, name))
        high_img = load_image(os.path.join(high_dir, name))
        
        axes[idx][0].imshow(low_img)
        axes[idx][0].set_title(f'Low-Light: {name}')
        axes[idx][0].axis('off')
        
        axes[idx][1].imshow(high_img)
        axes[idx][1].set_title(f'Normal-Light: {name}')
        axes[idx][1].axis('off')
    
    plt.tight_layout()
    plt.show()

print("=== Training Samples ===")
display_pairs(TRAIN_LOW, TRAIN_HIGH, num_pairs=5)

In [ ]:
print("=== Validation Samples ===")
display_pairs(VAL_LOW, VAL_HIGH, num_pairs=3)

## 4. Image Dimension Analysis

In [ ]:
def analyze_dimensions(directory, sample_size=50):
    """Analyze image dimensions in directory"""
    names = get_image_names(directory)
    if len(names) > sample_size:
        names = random.sample(names, sample_size)
    
    widths, heights = [], []
    for name in names:
        img = Image.open(os.path.join(directory, name))
        widths.append(img.width)
        heights.append(img.height)
    
    return {
        'min_w': min(widths), 'max_w': max(widths), 'mean_w': np.mean(widths),
        'min_h': min(heights), 'max_h': max(heights), 'mean_h': np.mean(heights),
        'unique_sizes': len(set(zip(widths, heights)))
    }

print("=== Image Dimensions ===")
train_dims = analyze_dimensions(TRAIN_LOW)
print(f"\nTraining Set:")
print(f"  Width:  min={train_dims['min_w']}, max={train_dims['max_w']}, mean={train_dims['mean_w']:.0f}")
print(f"  Height: min={train_dims['min_h']}, max={train_dims['max_h']}, mean={train_dims['mean_h']:.0f}")
print(f"  Unique sizes: {train_dims['unique_sizes']}")

val_dims = analyze_dimensions(VAL_LOW)
print(f"\nValidation Set:")
print(f"  Width:  min={val_dims['min_w']}, max={val_dims['max_w']}, mean={val_dims['mean_w']:.0f}")
print(f"  Height: min={val_dims['min_h']}, max={val_dims['max_h']}, mean={val_dims['mean_h']:.0f}")

## 5. Brightness Analysis

In [ ]:
def compute_brightness(directory, sample_size=30):
    """Compute mean brightness (luminance) of images"""
    names = get_image_names(directory)
    if len(names) > sample_size:
        names = random.sample(names, sample_size)
    
    brightness_values = []
    for name in names:
        img = np.array(Image.open(os.path.join(directory, name)).convert('RGB')) / 255.0
        # Luminance formula: 0.299*R + 0.587*G + 0.114*B
        luminance = 0.299 * img[:,:,0] + 0.587 * img[:,:,1] + 0.114 * img[:,:,2]
        brightness_values.append(luminance.mean())
    
    return brightness_values

# Compute brightness for low and high
low_brightness = compute_brightness(TRAIN_LOW)
high_brightness = compute_brightness(TRAIN_HIGH)

print("=== Brightness Analysis ===")
print(f"Low-light images:  mean={np.mean(low_brightness):.3f}, std={np.std(low_brightness):.3f}")
print(f"Normal-light images: mean={np.mean(high_brightness):.3f}, std={np.std(high_brightness):.3f}")
print(f"\nBrightness ratio (high/low): {np.mean(high_brightness)/np.mean(low_brightness):.2f}x")

# Plot histogram
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
ax.hist(low_brightness, bins=15, alpha=0.7, label='Low-Light', color='#333333')
ax.hist(high_brightness, bins=15, alpha=0.7, label='Normal-Light', color='#FFD700')
ax.set_xlabel('Mean Brightness')
ax.set_ylabel('Count')
ax.set_title('Brightness Distribution: Low vs Normal Light')
ax.legend()
plt.show()

## 6. Verify Shape Matching

In [ ]:
def verify_shape_matching(low_dir, high_dir, sample_size=50):
    """Verify that low and high pairs have matching shapes"""
    low_names = set(get_image_names(low_dir))
    high_names = set(get_image_names(high_dir))
    common = sorted(list(low_names.intersection(high_names)))
    
    if len(common) > sample_size:
        common = random.sample(common, sample_size)
    
    mismatched = []
    for name in common:
        low_img = Image.open(os.path.join(low_dir, name))
        high_img = Image.open(os.path.join(high_dir, name))
        
        if low_img.size != high_img.size:
            mismatched.append((name, low_img.size, high_img.size))
    
    return mismatched

print("=== Shape Verification ===")
mismatched = verify_shape_matching(TRAIN_LOW, TRAIN_HIGH)
if len(mismatched) == 0:
    print("All sampled pairs have matching shapes!")
else:
    print(f"Found {len(mismatched)} mismatched pairs:")
    for name, low_size, high_size in mismatched[:5]:
        print(f"  {name}: low={low_size}, high={high_size}")

## 7. Summary

**Key Findings:**
- Dataset is properly paired (same filenames in low/high directories)
- Images have varying dimensions - will need resizing for training
- Low-light images have ~3-5x lower brightness than normal
- All pairs have matching shapes

**Next Steps:**
1. Create PyTorch Dataset with proper transforms
2. Implement U-Net model architecture
3. Train with combined loss (L1 + Perceptual + SSIM)

In [ ]:
print("Dataset exploration complete!")
print("\nReady for model training.")